In [3]:
pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import os
from os.path import join, dirname
from dotenv import load_dotenv

load_dotenv(".env")

# Database Env
HOST = os.environ.get("HOST")
PORT = os.environ.get("PORT")
DB_USER = os.environ.get("DB_USER")
DB_PASSWORD = os.environ.get("DB_PASSWORD")
DB_SCHEMA = os.environ.get("DB_SCHEMA")

# Set app user and band
USER_ID = os.environ.get("USER_ID")
BAND_ID = os.environ.get("BAND_ID")

# Start and end dates for sets to update (leftmost and rightmost column header)
FIRST_SET = '7/4/2021'
LAST_SET = '9/17/2023'

In [7]:
pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import mysql.connector
import pandas as pd
import numpy as np

In [11]:
#data.csv has a particular format, with columns being sets and rows being songs:
# Most of the grid indicates which songs were in which sets (perhaps indicated by an X, a number, or some other text)
# Other columns like ARTIST and Notes give additional info on songs
 
# Song   ... ARTIST , Notes, ...  DATE_1, DATE_2, DATE_3,...
# Song_1 ... ArtistA, Blah , ...  X     ,       , 2     ,...
# Song_2 ... ArtistB, Yadda, ...  1     ,      X,       ,...

df = pd.read_csv("data.csv", sep=",", dtype=str,
                skipinitialspace=True, index_col="Song")

In [12]:
# Connect to server
def get_cnx():
    return mysql.connector.connect(
        host=HOST,
        port=PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_SCHEMA)

# Create songs in database with certain content

In [13]:
song_ids = {}
cnx = get_cnx()
try :
    for title, row in df.iterrows():
        
        # Get a cursor
        cur = cnx.cursor(prepared=True)
        # Check to see if song already exists
        cur.execute("""SELECT song_id FROM song
                        WHERE title = %s AND band_id = %s LIMIT 1""", (title,BAND_ID))
        rows = cur.fetchall()
        if rows:
            # save song_id
            song_ids[title] = rows[0][0]
            print(title, "already exists with song_id", rows[0][0])
        else :
            # Parse relevant data
            artist = row['ARTIST']
            if pd.isnull(artist):
                artist = ""
            notes = row['Notes']
            if pd.isnull(notes):
                notes = ""
            tags = []
    
            
            if "hymn" in artist.lower() or "hymn" in notes.lower():
                tags.append("Hymn")
            if "advent" in notes.lower():
                tags.append("Advent")
            if "christmas" in notes.lower():
                tags.append("Christmas")
            if "easter" in notes.lower():
                tags.append("Easter")
            if "lent" in notes.lower():
                tags.append("Lent")
            if "good friday" in notes.lower():
                tags.append("Good Friday")
            if "palm" in notes.lower():
                tags.append("Palm Sunday")
        
            # Execute a query to create the song
            cur.execute("""INSERT INTO song 
                        (title, artist, tags, notes, creator_id, band_id)
                        VALUES (%s, %s, %s, %s, %s, %s)""",
                       (title, artist, ",".join(tags), notes, USER_ID, BAND_ID))
            # Save the song_id for later
            song_ids[title] = cur.lastrowid
            print(title, "added with song_id", cur.lastrowid)

        # Make sure data is committed to the database
        cnx.commit()
        cur.close()
except Exception as e:
    print(repr(e))
finally:
    # Close connection
    cnx.close()

Cornerstone added with song_id 644
This I Believe (The Creed) added with song_id 645
Goodness of God added with song_id 646
Death Was Arrested added with song_id 647
Behold the Lamb (Communion) added with song_id 648
Great Are You Lord added with song_id 649
Dance Again added with song_id 650
The Lion and the Lamb added with song_id 651
Living Hope added with song_id 652
Way Maker added with song_id 653
Mighty To Save added with song_id 654
The Lord's Prayer (It's Yours) added with song_id 655
Build My Life added with song_id 656
In Christ Alone added with song_id 657
Shout to the Lord added with song_id 658
Take My Life (and Let It Be) added with song_id 659
I Love To Tell The Story added with song_id 660
Purify My Heart (Psalm 19) added with song_id 661
Lord I Need You added with song_id 662
The Solid Rock added with song_id 663
Your Will Be Done added with song_id 664
Everlasting God added with song_id 665
Indescribable added with song_id 666
God So Loved added with song_id 667
Hosa

# Create Sets and add songs within those sets.

In [14]:
import datetime
import calendar

# set_ids = {}
cnx = get_cnx()
try :
    for date in df.loc[:, FIRST_SET:LAST_SET]:
        month = int(date[:date.find("/")])
        day = int(date[date.find("/") + 1: date.rfind("/")])
        year = int(date[date.rfind("/") + 1:])
        intDay = datetime.date(year=year, month=month, day=day).weekday()
        dayOfTheWeek = calendar.day_name[intDay]

        
        # Get a cursor
        cur = cnx.cursor(prepared=True)
        # Check to see if song already exists
        cur.execute("""SELECT setlist_id FROM setlist
            WHERE date = STR_TO_DATE(%s, '%m/%d/%Y') AND band_id = %s LIMIT 1""", (date,BAND_ID))
        rows = cur.fetchall()
        if rows:
            print(date, "already exists with setlist_id", rows[0][0])
            # set_ids[date] = rows[0][0]
        else :
            # Parse relevant data
            
    
            # Execute a query to create the set
            cur.execute("""INSERT INTO setlist 
                        (name, date, creator_id, band_id)
                        VALUES (%s, STR_TO_DATE(%s, '%m/%d/%Y'), %s, %s)""",
                       (dayOfTheWeek + " Service", date, USER_ID, BAND_ID))
            # Save the song_id for later
            setlist_id = cur.lastrowid
            # set_ids[date] = setlist_id
            print(date, "added with setlist_id", cur.lastrowid)

            # Add songs to the set
            set_col = df[date]
            songs = set_col[set_col.notnull()]
            songs_sorted = songs.sort_values(ascending=True)
            for song, song_set_value in songs_sorted.items():
                song_id = song_ids[song]
                print ("\t", song, song_id, song_set_value)
                cur.execute("""INSERT INTO setlist_song 
                                (setlist_id, setlist_order, song_id, note)
                                select ?, COUNT(*), ?, ? from setlist_song where setlist_id = ?""",
                                   (setlist_id, song_id, song_set_value, setlist_id))
            
        # Make sure data is committed to the database
        cnx.commit()
        cur.close()
except Exception as e:
    raise e
finally:
    # Close connection
    cnx.close()



7/4/2021 added with setlist_id 175
	 Jesus Paid it All 698 x
	 O Come to the Altar 699 x
	 Never See the End 706 x
	 What A Beautiful Name 719 x
7/11/2021 added with setlist_id 176
	 Death Was Arrested 647 x
	 Living Hope 652 x
	 Lord I Need You 662 x
	 By Thy Mercy 798 x
7/18/2021 added with setlist_id 177
	 Our God Is With Us 685 x
	 Forever Reign 686 x
	 Great I Am 797 x
7/25/2021 added with setlist_id 178
	 He Will Hold Me Fast 712 x
	 Glorious Day 795 x
	 Through It All (hymn) 796 x
8/1/2021 added with setlist_id 179
	 How Deep The Father's Love 702 x
	 Come Thou Fount 704 x
	 Reckless Love 713 x
	 Who You Say I Am 728 x
8/8/2021 added with setlist_id 180
	 Everlasting God 665 x
	 From The Inside Out 722 x
	 The Potter's Hand 779 x
8/15/2021 added with setlist_id 181
	 Blessed Be Your Name 707 x
	 Hosanna (Praise Is Rising) 718 x
	 What A Beautiful Name 719 x
8/22/2021 added with setlist_id 182
	 Our God Is With Us 685 x
	 This Time I Will Bring Praise 705 x
	 Great Is Thy Faithfu